In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error
from math import sqrt
from collections import defaultdict

song_df_1 = pd.read_table('kaggle_visible_evaluation_triplets.txt',header=None)
song_df_1.columns = ['user_id', 'song_id', 'listen_count']
song_df_1.to_csv('user_song_listen.csv')


songs_metadata_file = 'song_data.csv'
song_df_2 = pd.read_csv(songs_metadata_file)
song_df = pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left")
song_df_2.head()


song_df['song'] = song_df['title'].map(str) + " - " + song_df['artist_name']
song_df.head()


,user_id,song_id,listen_count,title,release,artist_name,year,song
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1,You're The One,If There Was A Way,Dwight Yoakam,1990,You're The One - Dwight Yoakam
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1,Horn Concerto No. 4 in E flat K495: II. Romanc...,Mozart - Eine kleine Nachtmusik,Barry Tuckwell/Academy of St Martin-in-the-Fie...,0,Horn Concerto No. 4 in E flat K495: II. Romanc...
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1,Tive Sim,Nova Bis-Cartola,Cartola,1974,Tive Sim - Cartola
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1,Catch You Baby (Steve Pitron & Max Sanna Radio...,Catch You Baby,Lonnie Gordon,0,Catch You Baby (Steve Pitron & Max Sanna Radio...
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1,El Cuatrero,From Argentina To The World,Miguel Calo,2001,El Cuatrero - Miguel Calo


In [2]:
TRAING = True

from sklearn.model_selection import train_test_split

def dataTransform(dataname):
    if TRAING:
        song = song_df.head(10000)
    else:
        song = song_df
    

    user2index = defaultdict(int)
    disusers = list(set(song.user_id))
    for i in range(len(disusers)):
        user2index[disusers[i]] = i+1
    n_users = len(user2index.keys())
    print('We have ' + str(n_users) + ' users!!!')
     
    id2idx = defaultdict(int)
    disong = list(set(song.song_id))
    for i in range(len(disong)):
        id2idx[disong[i]] = i+1
    n_items = len(id2idx.keys())
    print('We have ' + str(n_items) +' songs')
    idx2id = {v:k for k,v in id2idx.items()}
    
    id2song = defaultdict(int)
    for i in range(song.shape[0]):
        id2song[song.song_id[i]] = song.song[i]

    song2id = {v:k for k,v in id2song.items()}

    song.song_id = song.song_id.map(id2idx)
    song.user_id = song.user_id.map(user2index)
 
    ratings = np.zeros((n_users, n_items))
    for row in song.itertuples():
        ratings[row[1]-1, row[2]-1] = row[3]

        
    
    sparsity = float(len(ratings.nonzero()[0]))
    sparsity /= (ratings.shape[0] * ratings.shape[1])
    sparsity *= 100
    print('Sparsity: {:4.2f}%'.format(sparsity))
    #print(song.head())
    train_data, test_data = train_test_split(song, test_size=0.25)

    train_data_matrix = np.zeros((n_users, n_items))
    #print(train_data_matrix.shape)
    test_data_matrix = np.zeros((n_users, n_items))
    
    for line in train_data.itertuples():
        #print(line[1])
        train_data_matrix[line[1]-1, line[2]-1] = line[3]  


    for line in test_data.itertuples():
        test_data_matrix[line[1]-1, line[2]-1] = line[3]
    
    print('Tranform Complete !!!!')
    return song,train_data_matrix,test_data_matrix,idx2id,id2idx,song2id,id2song
    
song,train_mat,test_mat,idx2id,id2idx,song2id,id2song = dataTransform(dataname = 'user_song_listen.csv')





We have 753 users!!!
We have 7399 songs


/Users/marcowang/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Sparsity: 0.18%
Tranform Complete !!!!


### This means that 0.18% of the user-item ratings have a value. Note that, although we filled in missing ratings as 0, we should not assume these values to truly be zero. More appropriately, they are just empty entries. We will split our data into training and test sets 


In [3]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [4]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_mat, metric='cosine')
item_similarity = pairwise_distances(train_mat.T, metric='cosine')

In [ ]:
item_prediction = predict(train_mat, item_similarity, type='item')
user_prediction = predict(train_mat, user_similarity, type='user')

print('User-based CF MSE: ' + str(get_mse(user_prediction, test_mat)))
print('Item-based CF MSE: ' + str(get_mse(item_prediction, test_mat)))

User-based CF MSE: 65.76645379295519
Item-based CF MSE: 65.81980510657758


In [ ]:
def predict_topk_nobias(ratings, similarity, kind='user', k=40):
    pred = np.zeros(ratings.shape)
    if kind == 'user':
        user_bias = ratings.mean(axis=1)
        ratings = (ratings - user_bias[:, np.newaxis]).copy()
        for i in range(ratings.shape[0]):
            top_k_users = [np.argsort(similarity[:,i])[:-k-1:-1]]
            for j in range(ratings.shape[1]):
                pred[i, j] = similarity[i, :][top_k_users].dot(ratings[:, j][top_k_users]) 
                pred[i, j] /= np.sum(np.abs(similarity[i, :][top_k_users]))
        pred += user_bias[:, np.newaxis]
    if kind == 'item':
        item_bias = ratings.mean(axis=0)
        ratings = (ratings - item_bias[np.newaxis, :]).copy()
        for j in range(ratings.shape[1]):
            top_k_items = [np.argsort(similarity[:,j])[:-k-1:-1]]
            for i in range(ratings.shape[0]):
                pred[i, j] = similarity[j, :][top_k_items].dot(ratings[i, :][top_k_items].T) 
                pred[i, j] /= np.sum(np.abs(similarity[j, :][top_k_items])) 
        pred += item_bias[np.newaxis, :]
        
    return pred

In [ ]:
k_array = [5, 15, 30, 50, 100, 200]
user_train_mse = []
user_test_mse = []
item_test_mse = []
item_train_mse = []

for k in k_array:
    user_pred = predict_topk_nobias(train_mat, user_similarity, kind='user', k=k)
    item_pred = predict_topk_nobias(train_mat, item_similarity, kind='item', k=k)
    
    user_train_mse += [get_mse(user_pred, train_mat)]
    user_test_mse += [get_mse(user_pred, test_mat)]
    
    item_train_mse += [get_mse(item_pred, train_mat)]
    item_test_mse += [get_mse(item_pred, test_mat)]  

/Users/marcowang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if __name__ == '__main__':
/Users/marcowang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # Remove the CWD from sys.path while we load stuff.
/Users/marcowang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

pal = sns.color_palette("Set2", 2)

plt.figure(figsize=(8, 6),dpi =300)
plt.plot(k_array, user_train_mse, c=pal[0], label='User-based train', alpha=0.5, linewidth=5)
plt.plot(k_array, user_test_mse, c=pal[0], label='User-based test', linewidth=5)
plt.plot(k_array, item_train_mse, c=pal[1], label='Item-based train', alpha=0.5, linewidth=5)
plt.plot(k_array, item_test_mse, c=pal[1], label='Item-based test', linewidth=5)
plt.legend(loc='best', fontsize=20)
plt.xticks(fontsize=16);
plt.yticks(fontsize=16);
plt.xlabel('k', fontsize=30);
plt.ylabel('MSE', fontsize=30)
plt.show()

In [ ]:
# item based 
def getName(id_ = 10):
    return id2song[idx2songs[id_]]
print(getName(6))



def top_k_song(similarity, mapper, song_idx, k=6):
    
    
    return [id2song[song] for song in [mapper[x] for x in np.argsort(similarity[song_idx,:])[:-k-1:-1]]]

songs = top_k_song(item_similarity, idx2id,song_idx = 9,k = 10)
songs






In [ ]:
name = input()
try:
    id_ = song2id.get(str(name))
    idx = int(id2idx[id_])
    
    answer = top_k_song(item_similarity, idx2id,song_idx = idx,k = 10)
    for i,v in enumerate(answer):
        print(""*80)
        print(str(i)+'. ',v)
except:
    print('We dont have this song in our database')
    
─=≡Σ(((つ•̀ω•́)つ    
    ♫꒰･◡･๑꒱
                                                                                 ♪└|∵|┐♪└|∵|┘♪┌|∵|┘♪
    
    
    
